In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['processed_data.csv']


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("processed_data.csv", index_col=0)

In [4]:
df.head()

,text,cs.CV,cs.LG,cs.AI,stat.ML,cs.CL,cs.NE,cs.IR,math.OC,cs.RO
0,We propose an architecture for VQA which utili...,1,0,1,1,1,1,0,0,0
1,Recent approaches based on artificial neural n...,0,1,1,1,1,1,0,0,0
2,We introduce the multiresolution recurrent neu...,0,1,1,1,1,1,0,0,0
3,Multi-task learning is motivated by the observ...,0,1,1,1,1,1,0,0,0
4,We present MILABOT: a deep reinforcement learn...,0,1,1,1,1,1,0,0,0


In [5]:
X = df["text"].values
y = df.drop(columns="text").values 

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
from keras.preprocessing import sequence
from keras.models import Model, Input
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam

In [19]:
max_features = 20000  # number of words we want to keep
maxlen = 300  # max length of the comments in the model
batch_size = 64  # batch size for the model
embedding_dims = 20  # dimension of the hidden variable, i.e. the embedding dimension

In [20]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
x_train = tokenizer.texts_to_sequences(X_train)
x_test = tokenizer.texts_to_sequences(X_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

32800 train sequences
8200 test sequences
Average train sequence length: 162
Average test sequence length: 163


In [21]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (32800, 300)
x_test shape: (8200, 300)


In [16]:
comment_emb.shape

TensorShape([Dimension(None), Dimension(100), Dimension(20)])

In [22]:
x_train[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
         168,     1,    76,     2,   586,   166,     1,   379,     2,
         842,   166,     9,   709,     4,   312,   160,    23,  1031,
           1,  1753,   592,   367,    21,   336,   138,   191,   591,
         771,     1,   160,    27,   172,   151,   333,     3,  1423,
         161,   730,    22,   126,     3,    47,   470,    58,     2,
           1,    81,    45,   103,    46,    39,    47,  8853,    19,
           1,   998,    49,  1235,    93,     1,    54,  3353,     2,
          46,    39,

In [84]:
comment_input = Input((maxlen,))

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
comment_emb = Embedding(max_features, 
                        embedding_dims, 
                        input_length=maxlen)(comment_input)
cnn1 = Conv1D(filters=50, kernel_size=5, activation='relu')(comment_emb)
# we add a GlobalMaxPooling1D, which will extract features from the embeddings
# of all words in the comment
h = GlobalMaxPooling1D()(cnn1)

# We project onto a six-unit output layer, and squash it with a sigmoid:
fc1 = Dense(200, activation='relu')(h)
d1 = Dropout(0.4)(fc1)
output = Dense(9, activation='sigmoid')(d1)

model = Model(inputs=comment_input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy', 
                      'categorical_accuracy'])


In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.1)


Train on 29520 samples, validate on 3280 samples
Epoch 1/10
 3136/29520 [==>...........................] - ETA: 4s - loss: 0.1690 - acc: 0.9331 - categorical_accuracy: 0.7567

In [86]:
sample = 9
print(y_test[sample])
print(x_test.shape)
print(np.array(model.predict(x_test)[sample]>0.5).astype(int))
y_pred = (model.predict(x_test)>0.5).astype(int)

[0 0 1 0 0 0 0 0 0]
(8200, 300)
[0 0 1 0 0 0 0 0 0]


In [80]:
from sklearn.metrics import accuracy_score
import keras 

In [ ]:
keras.backend.print_tensor(keras.metrics.top_k_categorical_accuracy(y_test, y_pred, k = 2))